In [ ]:
# streamlit/app.py
import streamlit as st
import pandas as pd
import geopandas as gpd
import os
from utils import get_engine
import plotly.express as px

st.set_page_config("Waste Tracker", layout="wide")
engine = get_engine()

@st.cache_data
def load_data():
    q = """
    SELECT t.date::date as date, l.kecamatan, SUM(f.volume) as volume
    FROM warehouse.fact_waste f
    JOIN warehouse.dim_time t ON f.time_id = t.id
    JOIN warehouse.dim_location l ON f.location_id = l.id
    GROUP BY t.date, l.kecamatan ORDER BY t.date;
    """
    return pd.read_sql(q, engine)

@st.cache_data
def load_geo():
    path = os.environ.get("WASTE_DATA_DIR", "./data")
    gfile = os.path.join(path, "kecamatan.geojson")
    if os.path.exists(gfile):
        gdf = gpd.read_file(gfile).to_crs(epsg=4326)

        # fix kolom!
        if "district" in gdf.columns:
            gdf['kecamatan'] = gdf['district'].astype(str).str.upper()
        else:
            gdf['kecamatan'] = gdf['kecamatan'].astype(str).str.upper()

        return gdf
    return None

st.title("Waste Tracker — Jakarta")

df = load_data()
if df.empty:
    st.warning("Tidak ada data di warehouse.fact_waste. Jalankan pipeline dulu.")
else:
    start = st.sidebar.date_input("Start", pd.to_datetime(df['date']).min().date())
    end = st.sidebar.date_input("End", pd.to_datetime(df['date']).max().date())
    ksel = st.sidebar.multiselect("Kecamatan", sorted(df['kecamatan'].unique()), default=None)
    mask = (pd.to_datetime(df['date']).dt.date >= start) & (pd.to_datetime(df['date']).dt.date <= end)
    if ksel:
        mask &= df['kecamatan'].isin(ksel)
    df2 = df[mask]

    st.metric("Total volume (selected)", f"{df2['volume'].sum():,.2f} ton")
    fig = px.line(df2.groupby('date')['volume'].sum().reset_index(), x='date', y='volume', title="Tren Volume")
    st.plotly_chart(fig, use_container_width=True)

    gdf = load_geo()
    if gdf is not None:
        agg = df2.groupby('kecamatan', as_index=False)['volume'].sum()
        merged = gdf.merge(agg, on='kecamatan', how='left').fillna(0)
        fig_map = px.choropleth_mapbox(merged, geojson=merged.__geo_interface__,
                                       locations=merged.index, color="volume",
                                       mapbox_style="open-street-map",
                                       center={"lat": -6.2, "lon": 106.8}, zoom=9)
        st.plotly_chart(fig_map, use_container_width=True)